# 2023-01-26 - 'portion' interval library for doing efficient interval calculations:

In [ ]:
from neuropy.core.epoch import Epoch
import portion as P
from neuropy.utils.efficient_interval_search import filter_epochs_by_speed
from neuropy.utils.efficient_interval_search import convert_PortionInterval_to_Epoch_obj, convert_Epoch_obj_to_PortionInterval_obj
# Filter *_replays_Interval by requiring them to be below the speed:
speed_thresh = 2.0
global_session = curr_active_pipeline.filtered_sessions['maze']
global_replays = Epoch(epochs=global_session.replay.epochs.get_valid_df())
speed_df = global_session.position.to_dataframe()
global_replays, above_speed_threshold_intervals, below_speed_threshold_intervals = filter_epochs_by_speed(speed_df, global_replays, speed_thresh=speed_thresh, debug_print=True)
# long_replays, short_replays, global_replays, above_speed_threshold_intervals, below_speed_threshold_intervals = filter_epochs_by_speed(speed_df, long_replays, short_replays, global_replays, speed_thresh=speed_thresh, debug_print=True)
global_ripple = global_session.ripple
global_pbe = global_session.pbe

In [ ]:
high_speed_epochs = convert_PortionInterval_to_Epoch_obj(above_speed_threshold_intervals)
low_speed_epochs = convert_PortionInterval_to_Epoch_obj(below_speed_threshold_intervals)

In [ ]:
## Plot them all on the epoch rect viewer:

import srsly
output_parent_folder = Path('C:/Users/pho/repos/PhoPy3DPositionAnalysis2021/data')
output_path = output_parent_folder.joinpath(f"{curr_active_pipeline.session_name}")
print(f'writing out to {output_path}')

In [ ]:
global_ripple.as_array()

In [ ]:
# srsly.write_gzip_json(path=output_path, data=dict(global_ripples=global_ripple., global_pbes=global_pbe, global_replays=global_replays))
# srsly.write_gzip_json(path=output_path, data=dict(global_ripples=global_ripple.to_dataframe(), global_pbes=global_pbe.to_dataframe(), global_replays=global_replays.to_dataframe()))
# global_ripple.to_dataframe().to

all_df = dict(global_ripples=global_ripple.to_dataframe(), global_pbes=global_pbe.to_dataframe(), global_replays=global_replays.to_dataframe())

In [ ]:
#consider all_df as a list of dataframes
with pd.HDFStore('df_store.h5') as df_store:
    for i in all_df.keys():
        df_store[i] = all_df[i]

In [ ]:
new_all_df = dict()
with pd.HDFStore('df_store.h5') as df_store:
    for i in df_store.keys():
        new_all_df[i] = df_store[i]

In [ ]:
# Store associated values with `P.IntervalDict()`:
d = P.IntervalDict()
d

In [ ]:
# i = 0
# for i in range(len(df)):
#     print(f'i: {i}')
#     print(f'df.index[i]: {df.index[i]}')
#     print(f"df.loc[i, 'speed']: {df.loc[i, 'speed']}")
#     print(f"df.loc[df.index[i], 'speed']: {df.loc[df.index[i], 'speed']}")
#     assert df.loc[df.index[i], 'speed'] == df.loc[i, 'speed']

# 2023-02-10 - _perform_spike_burst_detection_computation to get burst during replay events:

In [ ]:
%pdb on
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_spike_burst_detection_computation'], enabled_filter_names=['maze'], debug_print=True)

In [ ]:
active_burst_info = curr_active_pipeline.computation_results['maze'].computed_data['burst_detection']
active_burst_intervals = active_burst_info['burst_intervals']

In [ ]:
## Add the burst_detection burst_intervals to the active_2d_plot:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Render2DEventRectanglesHelper import Render2DEventRectanglesHelper
active_2d_plot = spike_raster_plt_2d
# active_burst_info = global_results['burst_detection']
# active_burst_intervals = active_burst_info['burst_intervals']
output_display_items = Render2DEventRectanglesHelper.add_event_rectangles(active_2d_plot, active_burst_intervals) # {'interval_rects_item': active_interval_rects_item}
active_interval_rects_item = output_display_items['interval_rects_item']
active_interval_rects_item

# 2023-02-08 - pynapple exploration and custom `write_neuroscope_intervals` function

In [ ]:
import os
import pynapple as nap
from pynapple.core.interval_set import IntervalSet

def write_neuroscope_intervals(isets, name, parent_path, basename, extension='derived.evt'):
    """Write events to load with neuroscope (e.g. ripples start and ends)
    
    Written 2023-02-08 to save computed replays out to .evt file to be read in by Neuroscope, but unfortunately it looks like the Epoch.to_neuroscope(...) function already works just as well (if not better) and doesn't require pynapple...

    Parameters
    ----------
    extension : str
        The extension of the file (e.g. basename.evt.py.rip)
    isets : IntervalSet
        The IntervalSet to write
    name : str
        The name of the events (e.g. Ripples)
    """
    start = isets.as_units("ms")["start"].values
    ends = isets.as_units("ms")["end"].values

    datatowrite = np.vstack((start, ends)).T.flatten()

    n = len(isets)

    texttowrite = np.vstack(
        (
            (np.repeat(np.array([name + " start"]), n)),
            (np.repeat(np.array([name + " end"]), n)),
        )
    ).T.flatten()

    evt_file = os.path.join(parent_path, basename + extension)

    f = open(evt_file, "w")
    for t, n in zip(datatowrite, texttowrite):
        f.writelines("{:1.6f}".format(t) + "\t" + n + "\n")
    f.close()

    return evt_file

outIntervalSet = IntervalSet(start=out.starts, end=out.stops, time_units="s")
write_neuroscope_intervals(outIntervalSet, 'pho_replays', str(active_sess.basepath), active_sess.name, extension='.PHO.evt')

In [ ]:
from pynapple.io import write_neuroscope_intervals
neurosuite

In [ ]:
nap_session = nap.load_session(str(active_sess.basepath), 'neurosuite')
nap_session

In [ ]:
active_sess.name

In [ ]:
active_sess.basepath

In [ ]:
parent_path = active_sess.basepath

# parent_path.with_suffix

In [ ]:
## Compute estimated replay intervals from the PBEs and export them to a neuroscope file
active_sess = curr_active_pipeline.filtered_sessions['maze']
out = active_sess.perform_compute_estimated_replay_epochs(min_epoch_included_duration=0.06, maximum_speed_thresh=2.0)
out.filename = active_sess.basepath.joinpath(active_sess.name).with_suffix('.derived')
final_export_path = out.to_neuroscope()
print(f'Exporting estimated replays to Neuroscope .evt file: {final_export_path}')

# 2023-02-08 - Automatic context using function decorators:

In [ ]:
_context_cache = active_sess.perform_compute_estimated_replay_epochs.__cache__()
_context_cache

In [ ]:
_context_cache.to_frame()

In [ ]:
def rebuild_arguments_from_stringmap_key(a_tuple_str):
    """ rebuilds the arg, kwargs from a cached function call that was cached using the keymap 'stringmap' """
    NULL = 'NULL'
    # each key is a tuple containing (args, kwargs) for the function
    # print(f'{a_tuple_str =}, len: {len(a_tuple_str)}')
    a_tuple = eval(a_tuple_str) # eval the string back into a real Python tuple
    args, kwargs = a_tuple
    # remove NULL entries
    kwargs = {k:v for k, v in kwargs.items() if v != 'NULL'}
    # print(kwargs)
    # print(f'{(args, kwargs) =}')
    return args, kwargs
    
_context_cache_recovered_args = {}

for a_key_str in _context_cache.keys():
    args, kwargs = rebuild_arguments_from_stringmap_key(a_key_str)
    _context_cache_recovered_args[a_key_str] = (args, kwargs)
    
_context_cache_recovered_args

Ideally could access all of the 'Epoch'-type members (or Epoch-convertable dataframes) the object (session in this case) has. This allows visualizations to "discover" the possible types of data they can visualize without having to code a custom menu-item/visualization/etc for each type of epoch. Ideally this like a "trait" or something, perhaps a property decorator?
Makes the most sense for computed data?

In [ ]:
active_sess.perform_compute_estimated_replay_epochs.dump()

In [ ]:
active_sess.perform_compute_estimated_replay_epochs.info()

In [ ]:
active_sess.perform_compute_estimated_replay_epochs.__map__()

In [ ]:
active_sess.perform_compute_estimated_replay_epochs.__mask__()